## Разведочный анализ данных для датасета Fashion MNIST.

Импортируем необходимые библиотеки

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import io
import plotly.express as px
from collections import Counter, defaultdict
import numpy as np
from IPython.display import display, HTML
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [28]:
labels = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

def load_image(image):
    return Image.open(io.BytesIO(image['bytes']))

def visualize_image(image, size=6):
    plt.figure(figsize=(size, size))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

def build_heatmap(pixels_dict, func = np.median, title="", size=28):
    result_arr = []
    for y in range(size):
        line_arr = []
        for x in range(size):
            line_arr.append(func(pixels_dict[x, y]))
        result_arr.append(line_arr)
    fig = px.imshow(result_arr, zmin=0, zmax=1, color_continuous_scale='gray', title=title)
    fig.update_layout(
        width=400,
        height=400,
        margin=dict(
            l=5,
            r=5,
            b=5,
            t=40,
            pad=4
        ),
        paper_bgcolor="LightSteelBlue",
    )
    return fig

def hheader(text, size=1):
    return display(HTML(f"<h{size}>{text}</h{size}>"))

Загрузим данные

In [18]:
splits = {'train': 'fashion_mnist/train-00000-of-00001.parquet', 'test': 'fashion_mnist/test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/zalando-datasets/fashion_mnist/" + splits["train"])
test_df = pd.read_parquet("hf://datasets/zalando-datasets/fashion_mnist/" + splits["test"])

Создадим для каждого изображения уникальный хэш, чтобы оценить, содержатся ли дубликаты или все данные уникальны в датасете.

In [19]:
train_df['named_label'] = train_df['label'].map(lambda x: labels[x])
train_df['loaded_image'] = train_df['image'].map(lambda x: load_image(x))
train_df['image_hash'] = train_df['image'].map(lambda x: hash(x['bytes']))


# test_df['named_label'] = test_df['label'].map(lambda x: labels[x])
train_df

,image,label,named_label,loaded_image,image_hash
0,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,9,Ankle boot,<PIL.PngImagePlugin.PngImageFile image mode=L ...,4845228445784961364
1,"{'bytes': b""\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...",0,T-shirt/top,<PIL.PngImagePlugin.PngImageFile image mode=L ...,-3104105095437305443
2,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,0,T-shirt/top,<PIL.PngImagePlugin.PngImageFile image mode=L ...,-899781993423124556
3,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,3,Dress,<PIL.PngImagePlugin.PngImageFile image mode=L ...,2386434958078699010
4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,0,T-shirt/top,<PIL.PngImagePlugin.PngImageFile image mode=L ...,7515191714391061894
...,...,...,...,...,...
59995,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,5,Sandal,<PIL.PngImagePlugin.PngImageFile image mode=L ...,8857482524727242077
59996,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,1,Trouser,<PIL.PngImagePlugin.PngImageFile image mode=L ...,3769294409867483582
59997,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,3,Dress,<PIL.PngImagePlugin.PngImageFile image mode=L ...,2954821560158303449
59998,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,0,T-shirt/top,<PIL.PngImagePlugin.PngImageFile image mode=L ...,1234733866344641642


В датасете все данные уникальны внутри каждой категории

In [26]:
hheader("Number of pictures in each category")
fig = px.histogram(train_df, x="named_label", text_auto=True)
fig.update_layout(
    bargap=0.2,
    title="Number pf pictures in each category",
    xaxis_title="Category",
    yaxis_title="Count",
    width=800,
    height=400
)
fig.show()

fig = px.histogram(train_df.groupby('named_label')['image_hash'].nunique().reset_index(), x="named_label", y='image_hash', text_auto=True)
fig.update_layout(
    bargap=0.2,
    title="Number of unique pictures in each category",
    xaxis_title="Category",
    yaxis_title="Count",
    width=800,
    height=400
)
fig.show()


Проверим, что изображения имеют одинаковый размер и соотношение сторон

In [27]:
sizes = []
for index, row in train_df.iterrows():
    image = row['loaded_image']
    x,y = image.size
    sizes.append(f"{x}x{y}")
size_count = Counter(sizes)
fig = px.histogram(x=size_count.keys(), y=size_count.values(), text_auto=True)
fig.update_layout(
    title="Pictures size distribution",
    xaxis_title="Picture size, pixels",
    yaxis_title="Count",
    width=800,
    height=400
)
fig

Построим хитмапу для всего датасета, на основе оценок ярости пискелей.

Будем использовать нормализацию значений яркости пикселей для повышения точности вычислений

In [22]:
pixel_dict = defaultdict(list)
category_pixel_dict = defaultdict(lambda: defaultdict(list))
for index, row in train_df.iterrows():
    image = row['loaded_image']
    width, height = image.size
    pixels = image.load()
    for y in range(height):
        for x in range(width):
            pixel = pixels[x, y] / 255
            pixel_dict[x,y].append(pixel)
            category_pixel_dict[row['label']][x, y].append(pixel)

In [29]:
hheader("Overall dataset heatmaps")
build_heatmap(pixel_dict, title="Overall heatmap with median values").show()
build_heatmap(pixel_dict, np.mean, title="Overall heatmap with mean values").show()

Построим хитмапу для каждой категории отдельно

In [30]:
hheader(f"Label specific heatmaps")
for label_id, label_name in labels.items():
    hheader(f"Heatmaps for {label_name}", 2)
    build_heatmap(category_pixel_dict[label_id], title="Median heatmaps").show()
    build_heatmap(category_pixel_dict[label_id], np.mean, title="Mean heatmaps").show()

Визуализируем результат кластеризации датасета, а также выведем статисктику по элементам каждого класса внутри кластеров.

In [31]:
hheader("Clustering")
images = [np.array(image).flatten() / 255 for image in train_df["loaded_image"]]
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(images)
sk_labels = kmeans.labels_
pca = PCA(n_components=2)
X_pca = pca.fit_transform(images)
clusted_df = pd.DataFrame(X_pca, columns=["PCA 1", "PCA 2"])
clusted_df['Cluster'] = sk_labels
fig = px.scatter(clusted_df, x="PCA 1", y="PCA 2", color="Cluster", title="K-means Clustering of Fashion MNIST",
                 color_continuous_scale='Viridis',
                 labels={"Cluster": "Cluster ID"}, width=800, height=800)
fig.update_traces(
    marker=dict(size=5, opacity=0.6),
)
fig.show()

hheader("Clusters research", 2)
cluster_to_orig = defaultdict(list)
for i, orig_label in enumerate(train_df["label"]):
    cluster_label = sk_labels[i]
    cluster_to_orig[int(cluster_label)].append(orig_label)

for k, v in sorted(cluster_to_orig.items()):
    full_dict = {lv: 0 for lk, lv in labels.items()}
    for orig_label, count in sorted(dict(Counter(v)).items()):
        full_dict[labels[orig_label]] += count
    fig = px.bar(x=full_dict.keys(), y=full_dict.values(), text_auto=True)
    fig.update_layout(
        bargap=0.2,
        title=f"Cluster {k}",
        xaxis_title="Category",
        yaxis_title="Count",
        width=800,
        height=400
    )
    fig.show()

Результат кластеризации с помощью K-means показывает, что:

1. кластеры пересекаются и не имеют четких границ. То есть векторные представления объектов разных классов в некоторой степени похожи.

В целом, ожидаемо, так как для одежды особенности объектов могут пересекаться, например, для футболок и кофт.

2. Кластеры сгруппированы в несколько областей, где некоторые кластеры преобладают.

Выводы, что могут быть проблемы с класстеризацией из-за структурной схожести некоторых классов.
Как мы видим, ни один кластер не соответствует одному классу одежды, а содержит смесь разных классов.